In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("Spark_Training").getOrCreate()

In [0]:
df = spark.read.csv("/FileStore/tables/wc2018_players.csv", inferSchema=True, header=True)

1.    Shows the names and height by adding 1 to the height column.
2.    shows the player name and simultaneously checks whether or not they have height >170
3.    Show FIFA Popular Name and 0 or 1 depending on Height>170
4.    name of  shortest player
5.    who is tallest of all. First we find the value of maximum height and then get the details of that player
6.    average height of the players in Argentina team.

In [0]:
df.show(10)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
|Argentina|  4|  DF|  ANSALDI Cristian|20.09.1986|   ANSALDI|     Torino FC (ITA)|   181|    73|
|Argentina|  5|  MF|      BIGLIA Lucas|30.01.1986|    BIGLIA|      AC Milan (ITA)|   175|    73|
|Argentina|  7|  MF|       BAN

In [0]:
df.select('FIFA Popular Name', 'Height').withColumn('Added Height', df['Height'] +1 ).show()

+------------------+------+------------+
| FIFA Popular Name|Height|Added Height|
+------------------+------+------------+
|TAGLIAFICO Nicolas|   169|         170|
|    PAVON Cristian|   169|         170|
|    LANZINI Manuel|   167|         168|
|    SALVIO Eduardo|   167|         168|
|      MESSI Lionel|   170|         171|
|  ANSALDI Cristian|   181|         182|
|      BIGLIA Lucas|   175|         176|
|       BANEGA Ever|   175|         176|
| MASCHERANO Javier|   174|         175|
|      DYBALA Paulo|   177|         178|
|     AGUERO Sergio|   172|         173|
|   HIGUAIN Gonzalo|   184|         185|
|    DI MARIA Angel|   178|         179|
|  LO CELSO Giovani|   177|         178|
|  MEZA Maximiliano|   180|         181|
|      ACUNA Marcos|   172|         173|
|CABALLERO Wilfredo|   186|         187|
|   MERCADO Gabriel|   181|         182|
|  OTAMENDI Nicolas|   181|         182|
|       ROJO Marcos|   189|         190|
+------------------+------+------------+
only showing top

In [0]:
#shows the player name and simultaneously checks whether or not they have height >170

In [0]:
df.filter(df['Height'] > 170).select('FIFA Popular Name').show()

+------------------+
| FIFA Popular Name|
+------------------+
|  ANSALDI Cristian|
|      BIGLIA Lucas|
|       BANEGA Ever|
| MASCHERANO Javier|
|      DYBALA Paulo|
|     AGUERO Sergio|
|   HIGUAIN Gonzalo|
|    DI MARIA Angel|
|  LO CELSO Giovani|
|  MEZA Maximiliano|
|      ACUNA Marcos|
|CABALLERO Wilfredo|
|   MERCADO Gabriel|
|  OTAMENDI Nicolas|
|       ROJO Marcos|
|    FAZIO Federico|
|     ARMANI Franco|
|     GUZMAN Nahuel|
|      KRUSE Robbie|
|    MEREDITH James|
+------------------+
only showing top 20 rows



In [0]:
#Show FIFA Popular Name and 0 or 1 depending on Height>170

In [0]:
from pyspark.sql.functions import regexp_replace
temp = df.withColumn('IsTaller', df['Height'] > 170).withColumn('isTaller', regexp_replace('IsTaller', 'true', '1'))
#df.withColumn('IsTaller', regexp_replace('isTaller', 'false', '0'))
temp.select('FIFA Popular Name', 'Height', 'isTaller').show()


+------------------+------+--------+
| FIFA Popular Name|Height|isTaller|
+------------------+------+--------+
|TAGLIAFICO Nicolas|   169|   false|
|    PAVON Cristian|   169|   false|
|    LANZINI Manuel|   167|   false|
|    SALVIO Eduardo|   167|   false|
|      MESSI Lionel|   170|   false|
|  ANSALDI Cristian|   181|       1|
|      BIGLIA Lucas|   175|       1|
|       BANEGA Ever|   175|       1|
| MASCHERANO Javier|   174|       1|
|      DYBALA Paulo|   177|       1|
|     AGUERO Sergio|   172|       1|
|   HIGUAIN Gonzalo|   184|       1|
|    DI MARIA Angel|   178|       1|
|  LO CELSO Giovani|   177|       1|
|  MEZA Maximiliano|   180|       1|
|      ACUNA Marcos|   172|       1|
|CABALLERO Wilfredo|   186|       1|
|   MERCADO Gabriel|   181|       1|
|  OTAMENDI Nicolas|   181|       1|
|       ROJO Marcos|   189|       1|
+------------------+------+--------+
only showing top 20 rows



In [0]:
#name of shortest player
df.agg({'Height' : 'min'}).show()
df.select("FIFA Popular Name").filter(df['Height'] == 165).show()
#df.select(min('Height')).show(

+-----------+
|min(Height)|
+-----------+
|        165|
+-----------+

+-----------------+
|FIFA Popular Name|
+-----------------+
| QUINTERO Alberto|
|   YAHIA ALSHEHRI|
|  SHAQIRI Xherdan|
+-----------------+



In [0]:
#who is tallest of all. First we find the value of maximum height and then get the details of that player
df.agg({"Height": "max"}).show()
df.select('FIFA Popular Name').filter(df["Height"] == 201).show()

+-----------+
|max(Height)|
+-----------+
|        201|
+-----------+

+-----------------+
|FIFA Popular Name|
+-----------------+
|    KALINIC Lovre|
+-----------------+



In [0]:
#average height of the players in Argentina team.
df.groupBy('Team').agg({'Height' : 'avg'}).filter(df['Team'] == 'Argentina').show()

+---------+------------------+
|     Team|       avg(Height)|
+---------+------------------+
|Argentina|178.43478260869566|
+---------+------------------+



Find the list of  oldest released movies.
How many movies are released each year?
How many number of movies are there for each rating?
How many users have rated each movie?
What is the total rating for each movie?
What is the average rating for each movie?

In [0]:
#Find the list of oldest released movies.
df = spark.read.csv("/FileStore/tables/movies.csv", inferSchema=True, header=True)


In [0]:
df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [0]:
df.createOrReplaceTempView('movies')
spark.sql("select * from movies").count()

Out[231]: 9125

In [0]:
df_with_dates_only = spark.sql("""select * from movies where title like '%(%%)' """)
df_with_dates_only.createOrReplaceTempView('movies')
df = spark.sql("""select *, substring(title,-5,4) as year from movies order by year""" )
df = df.where(df['year'] != '007-')
df.createOrReplaceTempView('movies')


In [0]:
spark.sql("""select min(year) from movies""").show()
spark.sql("""select * from movies where movies.year == 1902""").show()

+---------+
|min(year)|
+---------+
|     1902|
+---------+

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
|  32898|Trip to the Moon,...|Action|Adventure|...|1902|
+-------+--------------------+--------------------+----+



In [0]:
# How many movies are released each year?
from pyspark.sql.functions import sum,avg,max,min,mean,count
spark.sql("""select year,count(title) as numberOfMovies from movies GROUP BY movies.year Order by year""").show()


+----+--------------+
|year|numberOfMovies|
+----+--------------+
|1902|             1|
|1915|             1|
|1916|             2|
|1917|             1|
|1918|             1|
|1919|             1|
|1920|             3|
|1921|             5|
|1922|             6|
|1923|             3|
|1924|             5|
|1925|             8|
|1926|             6|
|1927|             8|
|1928|             5|
|1929|             7|
|1930|             5|
|1931|            14|
|1932|            14|
|1933|            13|
+----+--------------+
only showing top 20 rows



How many number of movies are there for each rating?

In [0]:
df1 = spark.read.csv("/FileStore/tables/ratings.csv", inferSchema=True, header=True)

In [0]:
df1.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows



In [0]:
df.show()

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
|  32898|Trip to the Moon,...|Action|Adventure|...|1902|
|   7065|Birth of a Nation...|           Drama|War|1915|
|   7243|Intolerance: Love...|               Drama|1916|
|  62383|20,000 Leagues Un...|Action|Adventure|...|1916|
|   8511|Immigrant, The (1...|              Comedy|1917|
|   3309|Dog's Life, A (1918)|              Comedy|1918|
|  72626|Billy Blazes, Esq...|      Comedy|Western|1919|
|   6987|Cabinet of Dr. Ca...|Crime|Fantasy|Horror|1920|
|  59945|Mark of Zorro, Th...|Adventure|Romance...|1920|
|  25737|Golem, The (Golem...|      Fantasy|Horror|1920|
|  83359|Play House, The (...|              Comedy|1921|
|  83096|Haunted House, Th...|              Comedy|1921|
|  83318|    Goat, The (1921)|              Comedy|1921|
|  83322|    Boat, The (1921)|              Comedy|1921|
|   3310|     Kid, The (1921)| 

In [0]:
df3 = df.join(df1, df.movieId == df1.movieId,'inner' )
df3.show()

+-------+--------------------+--------------------+----+------+-------+------+----------+
|movieId|               title|              genres|year|userId|movieId|rating| timestamp|
+-------+--------------------+--------------------+----+------+-------+------+----------+
|     31|Dangerous Minds (...|               Drama|1995|     1|     31|   2.5|1260759144|
|   1029|        Dumbo (1941)|Animation|Childre...|1941|     1|   1029|   3.0|1260759179|
|   1061|     Sleepers (1996)|            Thriller|1996|     1|   1061|   3.0|1260759182|
|   1129|Escape from New Y...|Action|Adventure|...|1981|     1|   1129|   2.0|1260759185|
|   1172|Cinema Paradiso (...|               Drama|1989|     1|   1172|   4.0|1260759205|
|   1263|Deer Hunter, The ...|           Drama|War|1978|     1|   1263|   2.0|1260759151|
|   1287|      Ben-Hur (1959)|Action|Adventure|...|1959|     1|   1287|   2.0|1260759187|
|   1293|       Gandhi (1982)|               Drama|1982|     1|   1293|   2.0|1260759148|
|   1339|D

In [0]:
df3.createOrReplaceTempView('movies')
spark.sql("""select rating,count(title) as numberOfMovies from movies GROUP BY movies.rating Order by rating""").show()

+------+--------------+
|rating|numberOfMovies|
+------+--------------+
|   0.5|          1100|
|   1.0|          3326|
|   1.5|          1687|
|   2.0|          7268|
|   2.5|          4447|
|   3.0|         20062|
|   3.5|         10538|
|   4.0|         28749|
|   4.5|          7720|
|   5.0|         15095|
+------+--------------+



How many users have rated each movie?

In [0]:
spark.sql("""select title,count(userId) as numberOfrating from movies GROUP BY movies.title Order by title""").show()

+--------------------+--------------+
|               title|numberOfrating|
+--------------------+--------------+
|        $9.99 (2008)|             3|
|'Hellboy': The Se...|             1|
|'Neath the Arizon...|             1|
|'Round Midnight (...|             2|
| 'Salem's Lot (2004)|             1|
|'Til There Was Yo...|             4|
|  'burbs, The (1989)|            19|
|'night Mother (1986)|             3|
|(500) Days of Sum...|            45|
|*batteries not in...|             7|
|...And God Spoke ...|             1|
|...And Justice fo...|            13|
|   1-900 (06) (1994)|             1|
|           10 (1979)|             5|
| 10 Attitudes (2001)|             1|
|10 Cloverfield La...|             5|
|10 Items or Less ...|             1|
|10 Things I Hate ...|            57|
|     10 Years (2011)|             1|
|    10,000 BC (2008)|             8|
+--------------------+--------------+
only showing top 20 rows



What is the total rating for each movie?

In [0]:
spark.sql("""select title,sum(rating) as numberOfrating from movies GROUP BY movies.title Order by title""").show()

+--------------------+--------------+
|               title|numberOfrating|
+--------------------+--------------+
|        $9.99 (2008)|          11.5|
|'Hellboy': The Se...|           2.0|
|'Neath the Arizon...|           0.5|
|'Round Midnight (...|           4.5|
| 'Salem's Lot (2004)|           3.5|
|'Til There Was Yo...|          10.5|
|  'burbs, The (1989)|          58.0|
|'night Mother (1986)|          15.0|
|(500) Days of Sum...|         169.0|
|*batteries not in...|          22.0|
|...And God Spoke ...|           1.0|
|...And Justice fo...|          48.0|
|   1-900 (06) (1994)|           4.0|
|           10 (1979)|          12.0|
| 10 Attitudes (2001)|           5.0|
|10 Cloverfield La...|          18.5|
|10 Items or Less ...|           3.5|
|10 Things I Hate ...|         198.0|
|     10 Years (2011)|           3.5|
|    10,000 BC (2008)|          12.0|
+--------------------+--------------+
only showing top 20 rows



What is the average rating for each movie?

In [0]:
spark.sql("""select title,mean(rating) as numberOfrating from movies GROUP BY movies.title Order by title""").show()

+--------------------+------------------+
|               title|    numberOfrating|
+--------------------+------------------+
|        $9.99 (2008)|3.8333333333333335|
|'Hellboy': The Se...|               2.0|
|'Neath the Arizon...|               0.5|
|'Round Midnight (...|              2.25|
| 'Salem's Lot (2004)|               3.5|
|'Til There Was Yo...|             2.625|
|  'burbs, The (1989)|3.0526315789473686|
|'night Mother (1986)|               5.0|
|(500) Days of Sum...|3.7555555555555555|
|*batteries not in...| 3.142857142857143|
|...And God Spoke ...|               1.0|
|...And Justice fo...|3.6923076923076925|
|   1-900 (06) (1994)|               4.0|
|           10 (1979)|               2.4|
| 10 Attitudes (2001)|               5.0|
|10 Cloverfield La...|               3.7|
|10 Items or Less ...|               3.5|
|10 Things I Hate ...| 3.473684210526316|
|     10 Years (2011)|               3.5|
|    10,000 BC (2008)|               1.5|
+--------------------+------------